# Station Outlier filter

This notebook shows the "station outlier filter" to detect... 

The original R code stems from https://github.com/LottedeVos/PWSQC/. 

Publication:
de Vos, L. W., Leijnse, H., Overeem, A., & Uijlenhoet, R. (2019). Quality control for crowdsourced personal weather stations to enable operational rainfall monitoring. _Geophysical Research Letters_, 46(15), 8820-8829.

The idea of the filter is to... 

In [1]:
# Import packages

import warnings

import numpy as np
import xarray as xr
import poligrain as plg
import matplotlib.pyplot as plt
import pandas as pd

import time

In [2]:
#ds_pws = xr.open_dataset('C:/Users/a002461/OPENSENSE/data/OpenSense_PWS_data_FZ_filtered_rainfall.nc')
ds_pws = xr.open_dataset('OpenSense_PWS_example_format_data.nc')

In [3]:
#slice to one month
ds_pws = ds_pws.sel(time = slice('2017-07-01','2017-07-31'))

## Calculate distance matrix

In [4]:
ds_pws.coords["x"], ds_pws.coords["y"] = plg.spatial.project_point_coordinates(
    x=ds_pws.longitude, y=ds_pws.latitude, target_projection="EPSG:25832"
)

In [5]:
distance_matrix = plg.spatial.calc_point_to_point_distances(ds_pws, ds_pws)

## Apply SO filter

In [6]:
# Set parameters
mint = 4032
mrain = 100
mmatch = 200
gamma = 0.15
beta = 0.2
n_stat = 2
max_distance = 10e3 # Boolean 2D data array, defining neihbours within max_distance for all stations
#neighbor_matrix = distance_matrix < max_distance

In [5]:
#so_flag = pypwsqc.flagging.so_filter(
#    ds_pws.rainfall,
#    ds_pws.nbrs_not_nan,
#    ds_pws.reference,
#    distance_matrix,
#    mint = 4032,
#    mrain = 100,
#    mmatch = 200,
#    gamma = 0.15,
#   beta = 0.2,
#    n_stat = 5,
#    max_distance = 10e3,
#)

In [7]:
# Default Bias Correction Factor
dbc = 1

In [8]:
# Boolean 2D array (id, time) of true (rain > 0) or false (rain =< 0) for all stations and timesteps
rainy_timesteps = ds_pws.rainfall > 0

In [9]:
# cumulative rainy timesteps per station
cum_rainy_timesteps = rainy_timesteps.cumsum(dim="time")

In [11]:
# initialize ugly solution
numbers = np.empty_like(ds_pws.rainfall)
numbers[:, :] = range(len(ds_pws.time))
numbers = numbers.astype(int)
ds_pws["number"] = (("id", "time"), numbers)

In [12]:
# initalize with empty numpy array
so_flag = np.ones_like(ds_pws.rainfall)
biascorrectiontable = np.ones_like(ds_pws.rainfall)

## Testing faster solution without for-loop over time

### one station only

In [ ]:
# loop through the stations (not time). For each station, slice first the whole data set to you period of interest, 
# then slice that data set to only the current station of interest. 
# This only gives nan in correlation ah well

In [ ]:
def flag_station_outlier_for_one_station(da_station, da_neighbors, window_length):
    s_station = da_station.to_series()
    s_neighbors = da_neighbors.to_series()
    
    corr = s_station.rolling(window_length, min_periods=1).corr(s_neighbors)
    n_rainy_timesteps = (s_neighbors > 0).rolling(window_length, min_periods=1).sum()
    #return corr
    
    import pandas as pd
    ds = xr.Dataset.from_dataframe(pd.DataFrame({'corr': corr}))
    ds['n_rainy_timesteps'] = xr.DataArray.from_series(n_rainy_timesteps)
    #ds['id'] = ds.id_neighbor.astype(str)
    return ds

In [ ]:
#ds_pws_t_slice = ds_pws.isel(time=slice(evaluation_period_start, evaluation_period_end))
ds_pws_t_slice = ds_pws.isel(time=slice(0, mint))

In [ ]:
ds_pws_station = ds_pws_t_slice.rainfall.sel(id=station_id)

In [24]:
#ds_pws_station: the dataset of the current station in question, in the last mint timesteps
# ds_neighbors.rainfall --> rainfall time series of the nbrs
# window lenght = I GUESS THIS IS MINT???
corr = flag_station_outlier_for_one_station(ds_pws_station, ds_neighbors.rainfall, window_length=4000)

In [31]:
# data set with correlation and nr rainy time steps with the ids of all nrbs
corr.corr.data

array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])

In [32]:
# loop through the stations (not time). For each station, slice first the whole data set to you period of interest, 
# then slice that data set to only the current station of interest. 

## first attempt with several stations

In [ ]:

    # picking stations within range, excluding itself
    neighbor_ids = distance_matrix.id.data[(distance_matrix.sel(id=pws_id) < max_distance) & (distance_matrix.sel(id=pws_id) > 0)]



In [39]:
%%time

max_distance = 10e3

corr_list = []
n_rainy_list = []

for i in range(len(ds_pws.id)):

    # create data set for station i
    ds_station = ds_pws.isel(id=i)
    pws_id = ds_station.id.values

    # one bias correction factor per station, iteratively updated per time step ??
    BCF_prev = dbc

    # picking stations within range, excluding itself
    
    neighbor_ids = distance_matrix.id.data[(distance_matrix.sel(id=pws_id) < max_distance) & (distance_matrix.sel(id=pws_id) > 0)]

    #ds_foo = flag_station_outlier_for_one_station(ds_station.rainfall, ds_neighbors.rainfall, window_length=1000)

    #corr_list.append(ds_foo.corr.median(dim='id_neighbor'))


['ams2' 'ams3' 'ams4' 'ams5' 'ams6' 'ams7' 'ams8' 'ams9' 'ams10' 'ams11'
 'ams12' 'ams13' 'ams14' 'ams15' 'ams16' 'ams17' 'ams19' 'ams20' 'ams21'
 'ams23' 'ams24' 'ams26']
['ams1' 'ams3' 'ams4' 'ams5' 'ams6' 'ams7' 'ams8' 'ams9' 'ams10' 'ams11'
 'ams12' 'ams13' 'ams14' 'ams15' 'ams16' 'ams17' 'ams19' 'ams20' 'ams21'
 'ams23' 'ams24' 'ams26' 'ams34']
['ams1' 'ams2' 'ams4' 'ams5' 'ams6' 'ams7' 'ams8' 'ams9' 'ams10' 'ams11'
 'ams12' 'ams13' 'ams14' 'ams15' 'ams16' 'ams17' 'ams18' 'ams19' 'ams20'
 'ams21' 'ams23' 'ams24' 'ams25' 'ams26' 'ams32']
['ams1' 'ams2' 'ams3' 'ams5' 'ams6' 'ams7' 'ams8' 'ams9' 'ams12' 'ams13'
 'ams14' 'ams16' 'ams17' 'ams18' 'ams19' 'ams20' 'ams23' 'ams25' 'ams27'
 'ams28' 'ams30' 'ams31' 'ams32' 'ams33']
['ams1' 'ams2' 'ams3' 'ams4' 'ams6' 'ams7' 'ams8' 'ams9' 'ams10' 'ams11'
 'ams12' 'ams13' 'ams14' 'ams15' 'ams16' 'ams17' 'ams18' 'ams19' 'ams20'
 'ams21' 'ams23' 'ams24' 'ams25' 'ams26' 'ams28' 'ams30' 'ams31' 'ams32']
['ams1' 'ams2' 'ams3' 'ams4' 'ams5' 'ams7' '

## Testing new way to select nbrs

In [52]:
%%time

for i, pws_id in enumerate(["ams45"]):  # enumerate(["ams1", "ams45", "ams17"]): enumerate(ds_pws.id.data): 

    # one bias correction factor per station, iteratively updated per time step
    BCF_prev = dbc

    # picking stations within range, excluding itself
    neighbor_ids = distance_matrix.id.data[(distance_matrix.sel(id=pws_id) < max_distance) & (distance_matrix.sel(id=pws_id) > 0)]

    # if there are not enough stations nearby, or no observations in the time series, all timesteps get flagged as -1
    if (len(neighbor_ids) < n_stat) or (
        ds_pws.rainfall.sel(id=pws_id).isnull().all()
    ):
        print(" SO filter cannot be applied for station", pws_id)
        so_flag[i, :] = -1


['ams14' 'ams16' 'ams17' 'ams18' 'ams19' 'ams22' 'ams23' 'ams24' 'ams25'
 'ams26' 'ams27' 'ams28' 'ams29' 'ams30' 'ams31' 'ams32' 'ams33' 'ams34'
 'ams35' 'ams36' 'ams37' 'ams38' 'ams39' 'ams40' 'ams41' 'ams42' 'ams43'
 'ams44' 'ams46' 'ams47' 'ams48' 'ams49' 'ams50' 'ams51' 'ams52' 'ams53'
 'ams54' 'ams55' 'ams56' 'ams57' 'ams58' 'ams59' 'ams60' 'ams61' 'ams62'
 'ams63' 'ams64' 'ams65' 'ams66' 'ams67' 'ams68' 'ams69' 'ams70' 'ams71'
 'ams72' 'ams73' 'ams74' 'ams75' 'ams76' 'ams77' 'ams78' 'ams79' 'ams80'
 'ams81' 'ams82' 'ams83' 'ams84' 'ams85' 'ams86' 'ams87' 'ams88' 'ams89'
 'ams90' 'ams91' 'ams92' 'ams93' 'ams94' 'ams95' 'ams96' 'ams97' 'ams98'
 'ams99' 'ams100' 'ams101' 'ams102' 'ams103' 'ams104' 'ams105' 'ams106'
 'ams107' 'ams108' 'ams109' 'ams110' 'ams111' 'ams112' 'ams113' 'ams114'
 'ams115' 'ams117' 'ams119' 'ams120' 'ams123' 'ams125' 'ams126']
filter cannot be applied for station ams45
CPU times: total: 31.2 ms
Wall time: 8.46 ms


## 25 sept 2024 continue below!

In [ ]:
    # if there are not enough stations nearby, or no observations in the time series, all timesteps get flagged as -1
    if (len(neighbors.id_neighbor) < n_stat) or (
        ds_pws.rainfall.sel(id=pws_id).isnull().all()
    ):
        so_flag[i, :] = -1

    else:
        for j, timestep in enumerate(
            ds_pws.time.isel(time=slice(mint, mint+5)).data, start=mint
        ):
            
            #print(timestep)
            #filter not applied in warmup period
            #if j < mint: ???
                #so_flag[i, j] = -1
                #biascorrectiontable[i,j] = BCF_prev
            #t1 = time.time()
            #if time.time() - tt > 30: # every now and then print some info
                #tt = time.time()
                #print('After 30 sec, station:', i, 'of', len(ds_pws.id), 'timestep:', j, 'of', len(ds_pws.time))
            
            # fixed evaluation period if at least mrain rainy timesteps in the last mint timesteps

            evaluation_period_start = j - mint +1
            evaluation_period_end = j

            # number of rainy timesteps within evaulation period from neighbors
            matches = (
                rainy_timesteps.sel(id=neighbors)
                .isel(time=slice(evaluation_period_start, evaluation_period_end))
                .sum()
                .data
            )

            if matches < mmatch:
            # filter cannot be applied if less than mmatch overlapping time intvervals
                so_flag[i, timestep] = -1

            # THE CODE ABOVE IS NOT THE PROBLEM - FAST COMPUTATION. WORK ON THE CODE BELOW. 

            else:
                # suppress runtime warning
                #print(i, 'filter can be applied')
                with warnings.catch_warnings():
                    warnings.simplefilter("ignore", category=RuntimeWarning)

                    corr_values = np.zeros_like(neighbors, dtype=float)
                    bias_values = np.zeros_like(neighbors, dtype=float)

                    # loop over neighbors
                    for k, neighbor_id in enumerate(neighbors.id_neighbor.data):
                        #print(k)
                        ds_pws_station = ds_pws.rainfall.sel(id=pws_id).isel(
                                time=slice(
                                    evaluation_period_start, evaluation_period_end
                                ))
                        ds_pws_nbr = ds_pws.rainfall.sel(id=neighbor_id).isel(
                                time=slice(
                                    evaluation_period_start, evaluation_period_end
                                )
                            )
                        # pearson correlation with neighboring stations
                        corr = xr.corr(ds_pws_station, ds_pws_nbr)
                        np.corrcoef(ds_pws_station.data, ds_pws_nbr)[0,1]
                        
                        corr_values[k] = corr.data  

                        # rain_pws - rain_reference (can be for example neighboring stations or radar)
                        delta_r = ds_pws_station -ds_pws_nbr 
                        bias = np.nanmean(delta_r) / np.nanmean(ds_pws_nbr)
                     
                        bias_values[k] = bias
                        
                        median_correlation = np.median(corr_values)
                        
                        # if median correlation is above threshold gamma,
                        # calculate new bias correction factor
                        if median_correlation > gamma:
                            #print('correlation is above threshold')
                            bias_values[np.isinf(bias_values)] = np.nan
                            median_bias = np.nanmedian(
                                bias_values
                            )  # median of bias of indivial bias to all neigbors
                            BCF_new = 1 / (1 + median_bias)
                            if np.abs(np.log(BCF_new / BCF_prev)) > np.log(1 + beta):
                                BCF_prev = BCF_new
                                biascorrectiontable[i,j] = BCF_new
                                #print('new BCF')

                        # if the median of the pearson correlation with the neighbors is too low, 
                        # it is a "station outlier"
                        else:
                            so_flag[i, j] = 1
                            biascorrectiontable[i,j] = BCF_prev
                            #print('below threshold, station outlier, previous BCF')
                        t2 = time.time()
            #print('timestep', j, 'took', t2 - t1, 'seconds to run')
    #so_flag.to_pickle('C:/Users/a002461/OPENSENSE/data/'+pws_id+'.pkl')      
    # initialize an empty list, append to the list, convert to xarray

## Only fixed evaluation period

In [36]:
#%load_ext line_profiler

In [29]:
%%time
ds_pws = ds_pws.load()
tt = time.time()

CPU times: total: 0 ns
Wall time: 5.01 ms


In [ ]:
%%time

for i, pws_id in enumerate(ds_pws.id.data):  # enumerate(["ams1", "ams45", "ams17"]):
    #print(pws_id)
    # one bias correction factor per station, iteratively updated per time step
    BCF_prev = dbc
    
    # picking stations within range, excluding itself
    nbrs_within_range = (distance_matrix.sel(id=pws_id) < max_distance) & (
        distance_matrix.sel(id=pws_id) > 0
    )

    # pick neighbors ids, drop all NanNs
    neighbors = (
        distance_matrix.sel(id=pws_id)
        .where(nbrs_within_range, drop=True)
        .id_neighbor
    )
    #print(neighbors.data)

    # if there are not enough stations nearby, or no observations in the time series, all timesteps get flagged as -1
    if (len(neighbors.id_neighbor) < n_stat) or (
        ds_pws.rainfall.sel(id=pws_id).isnull().all()
    ):
        so_flag[i, :] = -1

    else:
        for j, timestep in enumerate(
            ds_pws.time.isel(time=slice(mint, mint+5)).data, start=mint
        ):
            
            #print(timestep)
            #filter not applied in warmup period
            #if j < mint: ???
                #so_flag[i, j] = -1
                #biascorrectiontable[i,j] = BCF_prev
            #t1 = time.time()
            #if time.time() - tt > 30: # every now and then print some info
                #tt = time.time()
                #print('After 30 sec, station:', i, 'of', len(ds_pws.id), 'timestep:', j, 'of', len(ds_pws.time))
            
            # fixed evaluation period if at least mrain rainy timesteps in the last mint timesteps

            evaluation_period_start = j - mint +1
            evaluation_period_end = j

            # number of rainy timesteps within evaulation period from neighbors
            matches = (
                rainy_timesteps.sel(id=neighbors)
                .isel(time=slice(evaluation_period_start, evaluation_period_end))
                .sum()
                .data
            )

            if matches < mmatch:
            # filter cannot be applied if less than mmatch overlapping time intvervals
                so_flag[i, timestep] = -1

            # THE CODE ABOVE IS NOT THE PROBLEM - FAST COMPUTATION. WORK ON THE CODE BELOW. 

            else:
                # suppress runtime warning
                #print(i, 'filter can be applied')
                with warnings.catch_warnings():
                    warnings.simplefilter("ignore", category=RuntimeWarning)

                    corr_values = np.zeros_like(neighbors, dtype=float)
                    bias_values = np.zeros_like(neighbors, dtype=float)

                    # loop over neighbors
                    for k, neighbor_id in enumerate(neighbors.id_neighbor.data):
                        #print(k)
                        ds_pws_station = ds_pws.rainfall.sel(id=pws_id).isel(
                                time=slice(
                                    evaluation_period_start, evaluation_period_end
                                ))
                        ds_pws_nbr = ds_pws.rainfall.sel(id=neighbor_id).isel(
                                time=slice(
                                    evaluation_period_start, evaluation_period_end
                                )
                            )
                        # pearson correlation with neighboring stations
                        corr = xr.corr(ds_pws_station, ds_pws_nbr)
                        np.corrcoef(ds_pws_station.data, ds_pws_nbr)[0,1]
                        
                        corr_values[k] = corr.data  

                        # rain_pws - rain_reference (can be for example neighboring stations or radar)
                        delta_r = ds_pws_station -ds_pws_nbr 
                        bias = np.nanmean(delta_r) / np.nanmean(ds_pws_nbr)
                     
                        bias_values[k] = bias
                        
                        median_correlation = np.median(corr_values)
                        
                        # if median correlation is above threshold gamma,
                        # calculate new bias correction factor
                        if median_correlation > gamma:
                            #print('correlation is above threshold')
                            bias_values[np.isinf(bias_values)] = np.nan
                            median_bias = np.nanmedian(
                                bias_values
                            )  # median of bias of indivial bias to all neigbors
                            BCF_new = 1 / (1 + median_bias)
                            if np.abs(np.log(BCF_new / BCF_prev)) > np.log(1 + beta):
                                BCF_prev = BCF_new
                                biascorrectiontable[i,j] = BCF_new
                                #print('new BCF')

                        # if the median of the pearson correlation with the neighbors is too low, 
                        # it is a "station outlier"
                        else:
                            so_flag[i, j] = 1
                            biascorrectiontable[i,j] = BCF_prev
                            #print('below threshold, station outlier, previous BCF')
                        t2 = time.time()
            #print('timestep', j, 'took', t2 - t1, 'seconds to run')
    #so_flag.to_pickle('C:/Users/a002461/OPENSENSE/data/'+pws_id+'.pkl')      
    # initialize an empty list, append to the list, convert to xarray

## Cell below: variable and fixed evaluation period. I stopped the script after 4 hours, then it was still working on the first station.

In [14]:
%%time

for i, pws_id in enumerate(ds_pws.id.data):  # enumerate(["ams1", "ams45", "ams17"]):
    print(pws_id)
    # one bias correction factor per station, iteratively updated per time step
    BCF_prev = dbc
    
    # picking stations within range, excluding itself
    nbrs_within_range = (distance_matrix.sel(id=pws_id) < max_distance) & (
        distance_matrix.sel(id=pws_id) > 0
    )

    # pick neighbors ids, drop all NanNs
    neighbors = (
        distance_matrix.sel(id=pws_id)
        .where(nbrs_within_range, drop=True)
        .id_neighbor
    )

    # if there are not enough stations nearby, or no observations in the time series, all timesteps get flagged as -1
    if (len(neighbors.id_neighbor) < n_stat) or (
        ds_pws.rainfall.sel(id=pws_id).isnull().all()
    ):
        so_flag[i, :] = -1

    else:
        for j, timestep in enumerate(
            ds_pws.time.isel(time=slice(mint, -1)).data, start=mint
        ):
            #print(timestep)
            #filter not applied in warmup period
            #if j < mint: ???
                #so_flag[i, j] = -1
                #biascorrectiontable[i,j] = BCF_prev

            # fixed evaluation period if at least mrain rainy timesteps in the last mint timesteps
            if (
                cum_rainy_timesteps.sel(id=pws_id, time=timestep)
                - cum_rainy_timesteps.sel(id=pws_id).isel(time=j - mint + 1)
                >= mrain
            ):
                #print('fixed period')
                evaluation_period_start = j - mint +1
                evaluation_period_end = j

            # variable evaluation period if less than mrain rainy timesteps in the last mint timesteps (but before the considered period starts)
            elif (cum_rainy_timesteps.sel(id=pws_id).isel(time=j).data - mrain) > 0:
                start_timestamp = (
                    cum_rainy_timesteps.sel(id=pws_id)
                    .where(
                        cum_rainy_timesteps.sel(id=pws_id).data
                        == (
                            cum_rainy_timesteps.sel(id=pws_id).isel(time=j).data
                            - mrain
                        )
                    )
                    .dropna(dim="time")
                    .time.data
                )
                #print('variable period')
                #print(j)
                start_timestamp = start_timestamp[-1]

                evaluation_period_start = int(
                    ds_pws.number.sel(id=pws_id)
                    .where(
                        ds_pws.number.sel(id=pws_id).time.data == start_timestamp
                    )
                    .dropna(dim="time")
                    .data.item()
                )
                evaluation_period_end = j

            else:  
                # assign -1 to SO filter, we are in the 'warm up period' of the filter
                so_flag[i, j] = -1
                biascorrectiontable[i,j] = BCF_prev
                continue

            # number of rainy timesteps within evaulation period from neighbors
            matches = (
                rainy_timesteps.sel(id=neighbors)
                .isel(time=slice(evaluation_period_start, evaluation_period_end))
                .sum()
                .data
            )

            if matches < mmatch:
            #filter cannot be applied if less than mmatch overlapping time intvervals
                so_flag[i, timestep] = -1

            else:
                # suppress runtime warning
                with warnings.catch_warnings():
                    warnings.simplefilter("ignore", category=RuntimeWarning)

                    corr_values = np.zeros_like(neighbors, dtype=float)
                    bias_values = np.zeros_like(neighbors, dtype=float)

                    # loop over neighbors
                    for k, neighbor_id in enumerate(neighbors.id_neighbor.data):
                        
                        # pearson correlation with neighboring stations
                        corr = xr.corr(
                            ds_pws.rainfall.sel(id=pws_id).isel(
                                time=slice(
                                    evaluation_period_start, evaluation_period_end
                                )
                            ),
                            ds_pws.rainfall.sel(id=neighbor_id).isel(
                                time=slice(
                                    evaluation_period_start, evaluation_period_end
                                )
                            ),
                        )
                        corr_values[k] = corr.data  

                        # rain_pws - rain_reference (can be for example neighboring stations or radar)
                        delta_r = ds_pws.rainfall.sel(id=pws_id).isel(
                            time=slice(evaluation_period_start, evaluation_period_end)
                        ) - ds_pws.rainfall.sel(id=neighbor_id).isel(
                            time=slice(evaluation_period_start, evaluation_period_end)
                        )

                        bias = np.nanmean(delta_r) / np.nanmean(
                            ds_pws.rainfall.sel(id=neighbor_id).isel(
                                time=slice(
                                    evaluation_period_start, evaluation_period_end
                                )
                            )
                        )
                        bias_values[k] = bias
                        
                        median_correlation = np.median(corr_values)
                        
                        # if median correlation is above threshold gamma,
                        # calculate new bias correction factor
                        if median_correlation > gamma:
                            bias_values[np.isinf(bias_values)] = np.nan
                            median_bias = np.nanmedian(
                                bias_values
                            )  # median of bias of indivial bias to all neigbors
                            BCFnew = 1 / (1 + median_bias)
                            if np.abs(np.log(BCFnew / BCF_prev)) > np.log(1 + beta):
                                BCF_prev = BCFnew
                                biascorrectiontable[i,j] = BCF_new

                        # if the median of the pearson correlation with the neighbors is too low, 
                        # it is a "station outlier"
                        else:
                            so_flag[i, j] = 1
                            biascorrectiontable[i,j] = BCF_prev
    so_flag.to_pickle('C:/Users/a002461/OPENSENSE/data/'+pws_id+'.pkl')        


ams1


KeyboardInterrupt: 

In [ ]:
# add flags to dataset
ds_pws["so_flag"] = (("id", "time"), so_flag)
ds_pws["bias"] = (("id", "time"), biascorrectiontable)

In [ ]:
ds_pws.to_netcdf('C:/Users/a002461/OPENSENSE/data/OpenSense_PWS_example_format_data_SO_flags.nc')

## The code block below is not the problem; fast computation

In [35]:
%%time

for i, pws_id in enumerate(["ams1"]):  # enumerate(["ams1", "ams45", "ams17"]): enumerate(ds_pws.id.data): 
    #print(pws_id)
    # one bias correction factor per station, iteratively updated per time step
    BCF_prev = dbc
    
    # picking stations within range, excluding itself
    nbrs_within_range = (distance_matrix.sel(id=pws_id) < max_distance) & (
        distance_matrix.sel(id=pws_id) > 0
    )

    # pick neighbors ids, drop all NanNs
    neighbors = (
        distance_matrix.sel(id=pws_id)
        .where(nbrs_within_range, drop=True)
        .id_neighbor
    )
    #print(neighbors.data)

    # if there are not enough stations nearby, or no observations in the time series, all timesteps get flagged as -1
    if (len(neighbors.id_neighbor) < n_stat) or (
        ds_pws.rainfall.sel(id=pws_id).isnull().all()
    ):
        so_flag[i, :] = -1

    else:
        for j, timestep in enumerate(
            ds_pws.time.isel(time=slice(mint, mint+5)).data, start=mint
        ):
            
            #print(timestep)
            #filter not applied in warmup period
            #if j < mint: ???
                #so_flag[i, j] = -1
                #biascorrectiontable[i,j] = BCF_prev
            #t1 = time.time()
            #if time.time() - tt > 30: # every now and then print some info
                #tt = time.time()
                #print('After 30 sec, station:', i, 'of', len(ds_pws.id), 'timestep:', j, 'of', len(ds_pws.time))
            
            # fixed evaluation period if at least mrain rainy timesteps in the last mint timesteps

            evaluation_period_start = j - mint +1
            evaluation_period_end = j

            # number of rainy timesteps within evaulation period from neighbors
            matches = (
                rainy_timesteps.sel(id=neighbors)
                .isel(time=slice(evaluation_period_start, evaluation_period_end))
                .sum()
                .data
            )
            print(matches)

            if matches < mmatch:
            # filter cannot be applied if less than mmatch overlapping time intvervals
                so_flag[i, timestep] = -1

3677
3662
3646
3629
3612
CPU times: total: 15.6 ms
Wall time: 23.2 ms
